In [36]:
import pandas as pd
import numpy as np

import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go
import plotly.express as px
import os,sys
import dash_table
import flask
import dash
from dash.dependencies import Input, Output

data = pd.read_csv('data/OWID_energy/owid-energy-data.csv', sep = ',')
data = data[data.year >=1984 ]

import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

# How has the use of nuclear energy evolved over time? 
# How well does the use of nuclear energy correlate with changes in carbon emissions? 
# Are there characteristics of a country that correlate with increases or decreases in the use of nuclear energy?

In [37]:
#["fossil","gas","hydro","nuclear","oil","renewables","other_renewables","solar", "wind",]

# iso_code
# country
# year
# population

# Example variables
# wind_share_energy
# wind_consumption
# wind_energy_per_capita


In [38]:
# Some variables manipulation and pre-processing

# All variables of interest
variables = ["fossil" , "gas", "hydro", "nuclear" ,"oil" ,"wind" ,"renewables" ,"solar", "other_renewables"]

# Extract percentage of total energy from these sources
per_capita = [ v + "_energy_per_capita" for v in variables ]
data["total_energy"] = data[per_capita].sum(axis=1)
for v in variables:
    t = v + "_percent"
    data[t] = data[v + "_energy_per_capita"].divide(data["total_energy"]) * 100

# Update dataframe
data = data.round(decimals=1)



In [39]:
#data.head()

In [40]:
# Server definition
#app = dash.Dash(__name__ , external_stylesheets= [dbc.themes.CYBORG])
app = JupyterDash(__name__, external_stylesheets= [dbc.themes.CYBORG])

# Define the layout of the dashboard
app.layout = html.Div([

    dbc.Row(html.H1(
        className="title",
        children='Energy Source Interactive Dashboard', )
    ),
    
    html.Br(), # break line (empty space)

    dbc.Row([
        dbc.Col([
            dbc.Label("Electricity Type"),
            dcc.Dropdown(id="dropdown",
                         options=[
                             {"label": "Fossil", "value": "fossil"},
                             {"label": "Gas", "value": "gas"},
                             {"label": "Hydro", "value": "hydro"},
                             {"label": "Nuclear", "value": "nuclear"},
                             {"label": "Oil", "value": "oil"},
                             {"label": "Wind", "value": "wind"},
                             {"label": "Renewable", "value": "renewables"},
                             {"label": "Solar", "value": "solar"},
                             {"label": "Other Renewable", "value": "other_renewables"},

                         ],
                         multi=False,
                         value="nuclear",
                         style={'width': "50%"}
                         ),
        ]),

        dbc.Col([
            dbc.Label("Data Type"),
            dbc.RadioItems(
                id="radioitems-input",
                options=[
                    {"label": "Per Capita ", "value": "capita"},
                    {"label": "Percentage [%]", "value": "share"}, ],
                value="capita",
            ),
        ]),
    ]),

    html.Br(),

    html.Label("Year"),

    html.Br(),  # break line (empty space)

    dcc.Slider(id='slider',
                    marks={i: str(i) for i in range(1980, 2020)},
                    min=1985,
                    max=2019,
                    value=1985,
                    included=False,
                    updatemode='drag'

                    ),

    html.Br(),  # break line (empty space)

    # Placeholder for the maps and line plots
    dbc.Row([
        dbc.Col(
            html.Div([dcc.Graph(id='map', figure={},
                      clickData={'country': 'Austria'} )], className="figure-container"),
        ),
        dbc.Col(
            html.Div([dcc.Graph(id='line', figure={}, )], className="figure-container"),
        ),
    ])

],
    style={'width': '100%',
           'display': 'inline-block',
           'padding-left': '20px',
           'padding-right': '20px',
           },
)

In [41]:
# Map plot
@app.callback(
    Output(component_id='map',  component_property='figure'),
    Input(component_id='dropdown', component_property='value'),
    Input(component_id='slider', component_property='value'),
    Input(component_id='radioitems-input', component_property='value')

)

def make_map(variable, year, radio):

    # print('Year: ', year[1],
    #      'Variable: ' , variable )

    if radio == "capita":
      variable = variable + "_energy_per_capita"

    elif radio == "share":
        variable = variable + "_share_energy"

    # wind_share_energy
    # wind_consumption
    # wind_energy_per_capita
    # "_share_energy"
    # "_per_capita"

    mapp = data.loc [data.year == year ]

    print( np.nanmax(data[variable][:] ) )
    fig = go.Figure(data=go.Choropleth(
        locations = mapp['iso_code'],
        z = mapp[variable],
        text = mapp['country'],
        colorscale = 'Blues',
        zmax= np.nanmax(data[variable][:]),  #derived from maximum value of that variable across all data
        zmin=0,  # update

        #autocolorscale=False,
        #reversescale=True,
        #marker_line_color='darkgray',
        #marker_line_width=0.5,
        #colorbar_tickprefix = '$',
        #colorbar_title = 'GDP<br>Billions US$',
    ))

    ttl = variable.split("_")
    ttl = " ".join( ttl ).title()
    fig.update_layout(
        title_text= ttl ,
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        ),
    )


    fig.update_layout(
        margin=dict(l=10, r=10, t=30, b=10),
        paper_bgcolor="LightSteelBlue",
        autosize=False,
        width=600,
        height=300,
    )

    return fig


In [42]:
@app.callback(
    Output(component_id='line', component_property='figure'),
    Input('map', 'clickData'),
    Input(component_id='radioitems-input', component_property='value')

)

def make_line(clickData, radio):

    #print('CLICKDATA' , clickData )

    try:
        country = clickData['points'][0]['text']
    except:
        country = 'Austria'

    print('clickdata = Country = ', country )

    #print(data.head() )

    df = data.loc [data.country == country ]

    # Extract the variables for the time series plot as share or per capita
    if radio == "capita":
        vars = [ v + "_energy_per_capita" for v in variables ]
        ylab = "Energy per Capita [kWh]"
    elif radio == "share":
        vars = [v + "_percent" for v in variables]
        ylab = "Energy %"

    vars.append("year")

    dd = df[ vars ]

    #print(dd.head() )


    fig = px.line(dd, x='year', y=dd.columns)
    fig.update_layout(

        title_text=country,
        yaxis_title= ylab,

        margin=dict(l=10, r=10, t=30, b=10),
        autosize=False,
        width=600,
        height=300,
    )

    return fig

In [43]:
app.run_server( mode='inline')


24721.8
clickdata = Country =  Austria
clickdata = Country =  41.7
Austria
clickdata = Country =  Austria
24721.8
